In [2]:
import os
import cv2
import numpy as np

def load_data(base_dir):
    images = []
    masks = []
    
    # Traverse through each subdirectory
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.tif') and 'mask' not in filename:
                    img_path = os.path.join(folder_path, filename)
                    mask_path = os.path.join(folder_path, filename.replace('.tif', '_mask.tif'))
                    
                    # Check if corresponding mask exists
                    if os.path.exists(mask_path):
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                        
                        images.append(img)
                        masks.append(mask)
    
    return np.array(images), np.array(masks)

# Example usage
base_dir = r'C:\Users\Suraj\Downloads\Data\Data'
images, masks = load_data(base_dir)


In [4]:
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

images_clahe = np.array([apply_clahe(img) for img in images])


In [5]:
images_normalized = images_clahe / 255.0
masks_normalized = masks / 255.0

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def augment_data(images, masks):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    return datagen.flow(images[..., np.newaxis], masks[..., np.newaxis], batch_size=32)

augmented_data = augment_data(images_normalized, masks_normalized)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images_normalized, masks_normalized, test_size=0.2, random_state=42)


In [12]:
from tensorflow.keras import layers, Model

def nested_unet(input_shape):
    inputs = layers.Input(input_shape)
    
    # Encoder
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)
    
    output = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv2) 
    return Model(inputs, output)

model_nested = nested_unet((256, 256, 1))


In [16]:
model_nested.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [32]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

# Evaluate on test data
y_pred_nested = model_nested.predict(X_test)

dice_score_nested = dice_coefficient(y_test, y_pred_nested)

print(f'DICE Score Nested U-Net: {dice_score_nested}')


25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step


ValueError: operands could not be broadcast together with shapes (51511296,) (12877824,) 

In [28]:
y_pred_nested = np.squeeze(y_pred_nested)


In [29]:
import cv2
y_pred_resized = [cv2.resize(pred, (256, 256), interpolation=cv2.INTER_NEAREST) for pred in y_pred_nested]
y_pred_resized = np.array(y_pred_resized)


In [30]:
print(f"y_test shape: {y_test.shape}")
print(f"y_pred_nested shape: {y_pred_nested.shape}")


y_test shape: (786, 256, 256)
y_pred_nested shape: (786, 128, 128)
